In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
model =ChatOpenAI()

In [3]:
from langchain_community.document_loaders import WebBaseLoader
loader =WebBaseLoader("https://blog.langchain.dev/langgraph/")
docs=loader.load()

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter()
documetns=text_splitter.split_documents(docs)

In [6]:
documetns

[Document(page_content='LangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nCase Studies\n\n\n\n\nLangChain\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\nBy LangChain\n7 min read\nJan 17, 2024', metadata={'source': 'https://blog.langchain.dev/langgraph/', 'title': 'LangGraph', 'language': 'en'}),
 Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of a

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()
vector=FAISS.from_documents(documetns,embeddings)

In [9]:
retriever=vector.as_retriever()

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user","{input}"),
        ("user","Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
    ]
)

In [11]:
# creating Retriever Chain
from langchain.chains import create_history_aware_retriever
retriever_chain=create_history_aware_retriever(
    model,retriever,prompt
)

## chat gecmisi olustur

In [13]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history=[
    HumanMessage(
        content="Can I use LangGraph for agent runtimes?"
    ),
    AIMessage(content="Yes")
]

In [14]:
# gecmise göre bir soru sor
retriever_chain.invoke(
    {
        "chat_history":chat_history,
        "input":"Tell me how"
    }
)

[Document(page_content='LangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nCase Studies\n\n\n\n\nLangChain\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\nBy LangChain\n7 min read\nJan 17, 2024', metadata={'source': 'https://blog.langchain.dev/langgraph/', 'title': 'LangGraph', 'language': 'en'}),
 Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of a

## conversational Chain

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","Answer the user's questions based on the below context: \n\n{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user","{input}")]
)

In [17]:
from langchain.chains.combine_documents import create_stuff_documents_chain
documents_chain=create_stuff_documents_chain(model,prompt)

In [18]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever_chain,documents_chain)

### history

In [19]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history=[
    HumanMessage(
        content="Can I use LangGraph for agent runtimes?"
    ),
    AIMessage(content="Yes")
]

In [20]:
response=retrieval_chain.invoke(
    {
        "chat_history":chat_history,
        "input":"Tell me how"
    }
)

In [21]:
response["answer"]

"LangGraph is designed to facilitate the creation of agent runtimes by enabling the creation of cyclical graphs. Here's a brief overview of using LangGraph for agent runtimes:\n\n1. Define a StateGraph: Initialize a StateGraph by passing in a state definition. This state object is updated over time by nodes in the graph, returning operations to attributes of this state.\n\n2. Add Nodes: Add nodes to the graph using the `graph.add_node(name, value)` syntax. Nodes should be functions or LCEL runnables that accept a dictionary input in the same form as the State object and output a dictionary with keys of the State object to update.\n\n3. Add Edges: Connect nodes with edges. There are different types of edges: Starting Edge (connects the start of the graph to a node), Normal Edges (nodes called in sequence), and Conditional Edges (decision-making based on function output).\n\n4. Compile: Once the graph is defined, compile it into a runnable using `graph.compile()`. This runnable exposes m